# First chart data processing

In [1]:
import pandas as pd
import numpy as np

In [2]:
csv = pd.read_csv('../data/GlobalSentiment_Data.csv', index_col = 0)

In [3]:
dictionary = {}

In [6]:
for index, countries in csv.groupby('iso3'):
    if countries.shape[0] is not 152:
        display(index)

In [5]:
csv[csv['iso3'] == 'USA']

,iso3,date,senti_bert_std,senti_bert_std_trend,transit,drop_senti_bert,min_senti_bert_std,death,recovered,confirmed
20217,USA,2020-01-01,4.324215,1.336594,NaN,2020-03-05,2020-03-14,NaN,NaN,NaN
20218,USA,2020-01-02,0.168652,0.903590,NaN,2020-03-05,2020-03-14,NaN,NaN,NaN
20219,USA,2020-01-03,-1.049207,0.490647,NaN,2020-03-05,2020-03-14,NaN,NaN,NaN
20220,USA,2020-01-04,0.065922,0.088097,NaN,2020-03-05,2020-03-14,NaN,NaN,NaN
20221,USA,2020-01-05,-1.335513,-0.302358,NaN,2020-03-05,2020-03-14,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
20364,USA,2020-05-27,-0.703816,-0.759332,-12.605714,2020-03-05,2020-03-14,NaN,NaN,NaN
20365,USA,2020-05-28,-1.378175,-1.153662,-12.216936,2020-03-05,2020-03-14,NaN,NaN,NaN
20366,USA,2020-05-29,-1.440015,-1.553201,-11.385047,2020-03-05,2020-03-14,NaN,NaN,NaN
20367,USA,2020-05-30,-1.103954,-1.956014,-5.860681,2020-03-05,2020-03-14,NaN,NaN,NaN


In [7]:
# fix usa

confirmed_global_cases = pd.read_csv("../data/time_series_covid19_confirmed_global.csv")

In [8]:
us_cases = confirmed_global_cases[confirmed_global_cases['Country/Region'] == 'US'].drop(['Lat', 'Long','Country/Region','Province/State'], axis = 1).T.reset_index()

In [9]:
us_cases.columns = ['date', 'cases']

In [10]:
us_cases['date'] = pd.to_datetime(us_cases['date'])


In [30]:
new_df = csv.loc[(csv.iso3 == 'USA')]
new_df['date'] = pd.to_datetime(new_df['date'])

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [31]:
new_us_cases = (us_cases['cases']-us_cases.shift(1)['cases'].fillna(0))

In [32]:
new_us_cases

0           1.0
1           0.0
2           1.0
3           0.0
4           3.0
         ...   
624    100083.0
625    131142.0
626     27501.0
627     22194.0
628    116202.0
Name: cases, Length: 629, dtype: float64

In [43]:
_new_df = new_df.join(us_cases.set_index('date'), on='date', how="left")
_new_df.reset_index(inplace = True)
_new_df['confirmed'] = new_us_cases

In [49]:
_new_df.set_index('index', inplace = True)

In [50]:
 _new_df['confirmed']

index
20217        1.0
20218        0.0
20219        1.0
20220        0.0
20221        3.0
          ...   
20364    27215.0
20365    28401.0
20366    31685.0
20367    32534.0
20368    26067.0
Name: confirmed, Length: 152, dtype: float64

In [51]:
csv.loc[(csv.iso3 == 'USA'), 'confirmed'] = _new_df['confirmed']

In [52]:
csv.loc[(csv.iso3 == 'USA')]

,iso3,date,senti_bert_std,senti_bert_std_trend,transit,drop_senti_bert,min_senti_bert_std,death,recovered,confirmed
20217,USA,2020-01-01,4.324215,1.336594,NaN,2020-03-05,2020-03-14,NaN,NaN,1.0
20218,USA,2020-01-02,0.168652,0.903590,NaN,2020-03-05,2020-03-14,NaN,NaN,0.0
20219,USA,2020-01-03,-1.049207,0.490647,NaN,2020-03-05,2020-03-14,NaN,NaN,1.0
20220,USA,2020-01-04,0.065922,0.088097,NaN,2020-03-05,2020-03-14,NaN,NaN,0.0
20221,USA,2020-01-05,-1.335513,-0.302358,NaN,2020-03-05,2020-03-14,NaN,NaN,3.0
...,...,...,...,...,...,...,...,...,...,...
20364,USA,2020-05-27,-0.703816,-0.759332,-12.605714,2020-03-05,2020-03-14,NaN,NaN,27215.0
20365,USA,2020-05-28,-1.378175,-1.153662,-12.216936,2020-03-05,2020-03-14,NaN,NaN,28401.0
20366,USA,2020-05-29,-1.440015,-1.553201,-11.385047,2020-03-05,2020-03-14,NaN,NaN,31685.0
20367,USA,2020-05-30,-1.103954,-1.956014,-5.860681,2020-03-05,2020-03-14,NaN,NaN,32534.0


,senti_bert_trend,mean_transit,total_cases,iso3
date,,,,
2020-01-01,1.930908,NaN,NaN,Global
2020-01-02,1.471912,NaN,NaN,Global
2020-01-03,1.007502,NaN,NaN,Global
2020-01-04,0.528769,NaN,NaN,Global
2020-01-05,0.063586,NaN,NaN,Global
...,...,...,...,...
2020-05-27,0.562548,-60.222678,93990.0,Global
2020-05-28,0.353155,-59.910409,107055.0,Global
2020-05-29,0.142029,-59.653929,106747.0,Global


In [54]:
for index, countries in csv.groupby('iso3'):
    countries.sort_values('date', inplace=True)
    dictionary[index] = {
        'senti_bert_std': list(countries['senti_bert_std']),
        'senti_bert_std_trend': list(countries['senti_bert_std_trend']),
        'transit': list(countries['transit']),
        'drop_senti_bert': list(countries['drop_senti_bert']),
        'min_senti_bert_std': list(countries['min_senti_bert_std']),
        'death': list(countries['death']),
        'recovered': list(countries['recovered']),
        'dates': list(countries['date']),
        'confirmed': list(countries['confirmed'])
    }

In [55]:
import json
json_object = json.dumps(dictionary, indent = 4)   


In [56]:
with open("../data/GlobalSentiment_Data.json", "w") as outfile:  
    json.dump(dictionary, outfile) 

# Map data processing

In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd

We need to show a map with the following variables: shock_standard and HalfLife from the dataset provided.

First, we need to load the two files and merge them to have a geojson with countries' geometries and with the aditional information.

In [2]:
provided_data = pd.read_csv("../data/Final_Output.csv")
provided_data.head()

,iso3,country,continent,min_senti_bert,drop_senti_bert,averageN,HalfLife,shock,shock_standard
0,AFG,Afghanistan,Asia,3/13/2020,3/10/2020,448.256579,8.410348,NaN,NaN
1,AGO,Angola,Africa,3/23/2020,3/11/2020,817.013158,12.320138,-2.156,-0.660154
2,ALB,Albania,Europe,3/25/2020,3/7/2020,288.138158,2.666999,NaN,NaN
3,ARE,United Arab Emirates,Asia,3/15/2020,3/7/2020,17878.046050,48.739074,-0.849,-0.686110
4,ARG,Argentina,South America,3/16/2020,3/10/2020,104898.302600,27.957062,-2.045,-0.879744


In [4]:
countries_shapes = gpd.read_file("../data/countries.geojson")
countries_shapes.head()

,ADMIN,ISO_A3,ISO_A2,geometry
0,Aruba,ABW,AW,"MULTIPOLYGON (((-69.99694 12.57758, -69.93639 ..."
1,Afghanistan,AFG,AF,"MULTIPOLYGON (((71.04980 38.40866, 71.05714 38..."
2,Angola,AGO,AO,"MULTIPOLYGON (((11.73752 -16.69258, 11.73851 -..."
3,Anguilla,AIA,AI,"MULTIPOLYGON (((-63.03767 18.21296, -63.09952 ..."
4,Albania,ALB,AL,"MULTIPOLYGON (((19.74777 42.57890, 19.74601 42..."


Now, merge with ISO3 field.

In [5]:
countries_shapes = countries_shapes[countries_shapes['ISO_A3'] != '-99']

In [6]:
provided_data.drop([45], inplace=True)

In [7]:
provided_data[provided_data['iso3'] == 'GEO']

,iso3,country,continent,min_senti_bert,drop_senti_bert,averageN,HalfLife,shock,shock_standard
44,GEO,Georgia,Europe,3/13/2020,3/9/2020,333.611842,29.691186,NaN,NaN


In [11]:
merged_data = provided_data.merge(countries_shapes, right_on="ISO_A3", left_on="iso3", validate="1:1")
merged_data.head()

,iso3,country,continent,min_senti_bert,drop_senti_bert,averageN,HalfLife,shock,shock_standard,ADMIN,ISO_A3,ISO_A2,geometry
0,AFG,Afghanistan,Asia,3/13/2020,3/10/2020,448.256579,8.410348,NaN,NaN,Afghanistan,AFG,AF,"MULTIPOLYGON (((71.04980 38.40866, 71.05714 38..."
1,AGO,Angola,Africa,3/23/2020,3/11/2020,817.013158,12.320138,-2.156,-0.660154,Angola,AGO,AO,"MULTIPOLYGON (((11.73752 -16.69258, 11.73851 -..."
2,ALB,Albania,Europe,3/25/2020,3/7/2020,288.138158,2.666999,NaN,NaN,Albania,ALB,AL,"MULTIPOLYGON (((19.74777 42.57890, 19.74601 42..."
3,ARE,United Arab Emirates,Asia,3/15/2020,3/7/2020,17878.046050,48.739074,-0.849,-0.686110,United Arab Emirates,ARE,AE,"MULTIPOLYGON (((53.86305 24.23469, 53.88860 24..."
4,ARG,Argentina,South America,3/16/2020,3/10/2020,104898.302600,27.957062,-2.045,-0.879744,Argentina,ARG,AR,"MULTIPOLYGON (((-68.65412 -54.88624, -68.65414..."


In [12]:
merged_data.keys()

Index(['iso3', 'country', 'continent', 'min_senti_bert', 'drop_senti_bert',
       'averageN', 'HalfLife', 'shock', 'shock_standard', 'ADMIN', 'ISO_A3',
       'ISO_A2', 'geometry'],
      dtype='object')

Filter in case we need full variables

In [13]:
merged_data = merged_data[['iso3', 'country', 'continent', 'min_senti_bert', 'drop_senti_bert',
       'averageN', 'HalfLife', 'shock', 'shock_standard', 'geometry', 'ISO_A2']]

Filter to get only the variables that we have discussed over slack

In [14]:
merged_data = merged_data[['iso3', 'country', 'continent', 'HalfLife', 'shock_standard', 'geometry', 'ISO_A2']]

In [15]:
merged_data = gpd.GeoDataFrame(merged_data)

In [16]:
merged_data.to_file("../data/countries_with_data.geojson", driver="GeoJSON")

In [17]:
merged_data['shock_standard'].describe(percentiles = [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1])

count    117.000000
mean      -0.848760
std        0.584025
min       -3.307960
10%       -1.610268
20%       -1.212389
30%       -1.000052
40%       -0.868296
50%       -0.723554
60%       -0.621621
70%       -0.485392
80%       -0.388376
90%       -0.230394
100%      -0.007038
max       -0.007038
Name: shock_standard, dtype: float64